**NLP Homework 1**
*by Kharkhanova Ekaterina*





Используем данные из готового датасета с отзывами на мобильные телефоны

In [19]:
import pandas as pd
import nltk

df = pd.read_csv('/content/data.csv', error_bad_lines=False, engine="python")
df.head(100)

Skipping line 275521: unexpected end of data


,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5
...,...,...
95,"Ну что сказать)? Кому то по душе ios, кому то ...",4
96,Судите сами. После iPhone 4s взял sony Z1 comp...,5
97,Мой путь смартфонопользования начинался с винд...,5
98,"отличный телефон, больше всего радует стабильн...",5


Создаем тестовую выборку, чтобы использовать ее позже

In [20]:
positive_check_base = df[df.Rating==5].head(5)
positive_check_base

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
4,Достойный телефон. Пользоваться одно удовольст...,5
5,6s gold 64gb,5
6,Мой первый айфон. Скажу честно- Эппл ранее ник...,5
7,Мне очень понравилась эта модель! Во-первых кл...,5


In [21]:
negative_check_base = df[df.Rating==1].head(5)
negative_check_base

,Review,Rating
38,Очень сильно разочарован компанией SONY. Испор...,1
51,"Дизайн , конечно, хорош.",1
56,В первый год эксплуатации еще было все нормаль...,1
60,Зарекаюсь от продукции Асус. Смартфон невероят...,1
62,"В первые 2 недели все работало, спустя это вре...",1


In [22]:
positive_check_base = positive_check_base['Review'].tolist()
negative_check_base = negative_check_base['Review'].tolist()
check_base = positive_check_base + negative_check_base
print(check_base)
right_answers = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


['3D Touch просто восхитительная вещь! Заряд держит целый день. В розовом цвете смотрится очень необычно. Touch ID очень быстрый и удобный. Всем советую этот телефон!', 'Достойный телефон. Пользоваться одно удовольствие.', '6s gold 64gb', 'Мой первый айфон. Скажу честно- Эппл ранее никогда в руках не держала, хотелось попробовать что-то новое, андроид уж слишком приелся. Первое, что порадовало- это дизайн и качество сборки. Тут все на высоте. Постоянно любуюсь телефоном, но он без чехла скользкий. Дисплей шикарный. Яркий, насыщенный, все летает. Как будто попадаешь в другой мир. Батарейка-не знаю почему все жалуются. За неё, кстати, переживала. Ан нет, на день с головой хватает.  Камера очень хорошая, очень четкая, яркая, быстрая. Селфи камера тоже крутая, не шумит, хороший угол обзора. Live photo - как по мне бред) 3D Touch - пока непривычно, часто вместо нажатия сильнее просто зажимаю, но думаю что освою, правда потом будет наверно трудно андроид в руки взять (хотя может к тому момен

Создаем обучающую выборку

In [5]:
positive_train_base = df[df.Rating==5].tail(40)
positive_train_base = positive_train_base['Review'].tolist()
positive_train_base = ' '.join(positive_train_base)
positive_train_base

'купил летом 2014,цена и качество (7990) супер)конечно чёрную крышку бы ещё) батарею держит до упора,позднего вечера,лично у меня при умеренном использовании, без игр,фоток штук 30-40,gps целый день,инет в вк, ютубе часик. Хороший бюджетный телефон, полностью оправдывает свою недорогую цену. Имея множество достоинств, он также не обделен недостатками, как и другие телефоны. Но, в отличие от других дорогих телефонов, Lenovo сделал свой более аккуратным и производительным. Сколько бы я не играла, не мучила бы его, я не заметила никаких тормозов, подвисаний и тем более лагов. Все проблемы и недостатки, которые я прочитала в комментариях меня очень огорчили, т.к. не все это правда, особенно про Wi-Fi, может быть у Вас он не очень. Ну а в остальном все эти не столь значительные проблемы решаются: проигрывание музыки можно сделать приятным, если установить плеер, ну или в конце концов у многих в наше время есть отдельный плеер для музыки, плохую гарнитуру можно заменить, а чехол потрудиться 

In [6]:
negative_train_base = df[df.Rating==1].tail(40)
negative_train_base = negative_train_base['Review'].tolist()
negative_train_base = ' '.join(negative_train_base)
negative_train_base

'Это самый мерзкий телефон, который у меня был.\rПокупался в январе 16 года. Через месяца 2-3 телефон стал выключаться при потере GSM/LTE сетей, а включить его можно было только после подключения к питанию, будь то 220 или павербанка.\rПримерно в то же время перестала работать камера, запускаешь приложение - он пишет "приложение было остановлено" - никакие ребуты/резеты/сброс до заводских параметров не помог. обновления от sony тоже мимо, ничего не изменилось. Буквально пару дней назад, после 5-6 перезагрузок подряд, ввиду вышеописанной проблемы с потерей сетей, камера, о чудо! Стала работать, но телефон стал люто греться, даже если выключена подсветка/отключены лишние приложения и жрать батарею ~25�-6 минут. При чем, подключаешь его в розетку, а заряд батареи тратится быстрее, чем заряжается.\rТех.поддержка sony абсолютно ничем не помогла, сказали везти в их сервис и чинить за деньги, т.к. гарантия только 1 год. Что самое забавное они абсолютно никак не реагируют, что подобный глюк им

Preprocessing

In [7]:
!pip uninstall spacy

Found existing installation: spacy 2.2.4
Uninstalling spacy-2.2.4:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/bin/*
    /usr/local/lib/python3.7/dist-packages/spacy-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/bin/theano_cache.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.2.4


In [8]:
! pip install spacy==3.0.0 -q
! python -m spacy download ru_core_news_sm -q

     |████████████████████████████████| 12.7 MB 226 kB/s 
     |████████████████████████████████| 456 kB 38.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 623 kB 53.4 MB/s 
     |████████████████████████████████| 9.1 MB 12.8 MB/s 
     |████████████████████████████████| 17.9 MB 1.4 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [35]:
import spacy

nlp = spacy.load('ru_core_news_sm')
doc_p = nlp(positive_train_base.lower())

token_list_p = []
pos_tags = []
for token in doc_p:
  if token.is_alpha or token.like_num or not token.is_punct:
    token_list_p.append(token.lemma_)
    pos_tags.append(token.pos_) # new

# new
upd_list = []
for i, tag in enumerate(pos_tags):
  if tag == 'ADJ':
    if 'NOUN' in pos_tags[i + 1]:
      collocation = token_list_p[i] + ' ' + token_list_p[i + 1]
      upd_list.append(collocation)

upd_list_2 = []
for i, tag in enumerate(pos_tags):
  if tag == 'ADV':
    if 'VERB' in pos_tags[i + 1]:
      collocation = token_list_p[i] + ' ' + token_list_p[i + 1]
      upd_list_2.append(collocation)


new_list = token_list_p + upd_list + upd_list_2 
# end of new block

positive_tokens = set(new_list)
print(positive_tokens)


{'нормально', 'нет', '10', 'выпивать', 'возможность', 'отличный аккумулятор', 'лаконичный', 'инет', 'решить', 'правда', 'потеряю', 'время', 'как', 'первый смартфон', 'суперэкономного', 'китайский', 'полноценный', 'ощутимых', 'да', 'сомневаться', 'разница', 'запилить', 'сдержанно', 'калькулятор', 'чат', 'интерфейс', 'решаться', 'текстовый редактор', 'юзаю', 'глюк', 'пр', 'его', 'поискать', 'трудный', 'просто', 'жалеть', 'шучу', 'лишь', 'этот', 'цветной', 'умеренный', 'негативный', 'теоефон', 'дружелюбный', 'последний прошивках', 'надпись', 'значительно', 'неработает', 'т.к', 'здесь', 'всьо', 'кстати', 'плавненько', '472', 'халява', 'смарфон', 'общий сумма', 'нём', 'после', 'одна', 'а', 'ночь', 'неэстетично', 'камера', 'возможно', 'четко', 'зернистость', 'лаг', 'отлично рекомендовать', 'качественный', 'подвисаний', 'бюджетник', 'заказать', 'добротно', 'отличный соотношение', 'стоить', 'адекватный цена', 'хороший смартфон', 'девайс', 'оправдывать', 'он', 'второй', 'при', 'текст', 'чужой',

In [36]:
doc_n = nlp(negative_train_base.lower())
token_list_n = []
pos_tags = []
for token in doc_n:
  if token.is_alpha or token.like_num or not token.is_punct:
    token_list_n.append(token.lemma_)
    pos_tags.append(token.pos_) # new

# new
upd_list = []
for i, tag in enumerate(pos_tags):
  if tag == 'ADJ':
    if 'NOUN' in pos_tags[i + 1]:
      collocation = token_list_n[i] + ' ' + token_list_n[i + 1]
      upd_list.append(collocation)

upd_list_2 = []
for i, tag in enumerate(pos_tags):
  if tag == 'ADV':
    if 'VERB' in pos_tags[i + 1]:
      collocation = token_list_n[i] + ' ' + token_list_n[i + 1]
      upd_list_2.append(collocation)


new_list = token_list_n + upd_list + upd_list_2 
# end of new block

negative_tokens = set(new_list)
print(negative_tokens)

{'вообще', 'дишманскую', 'испортиться', 'нормально', 'дотронуться', 'нет', 'другое телефон', 'питание', 'срабатывать', '10', 'заканчивать', 'установиться', 'посоветуйте', 'лучше', 'устанавливаться', 'особо', 'инет', 'задний панель', 'мало подходить', 'решить', 'разве', 'старенький', 'следующий телефон', 'отказать', 'время', 'как', 'срываться', 'мимо', 'отключатся', 'компания', 'новый', 'пойти', 'случиться', 'да', 'вдруг начаться', 'разница', 'короче говорить', 'пытать', 'вечно', 'надежда', 'оператор', 'прикладывание', 'нерабочий аппарат', 'глюк', 'однако', 'его', 'подряд', 'просто', 'каждый', 'нажимать', 'снова', 'слабо', 'мною', 'жалеть', 'родной камера', 'этот', 'негативный', 'втюхивают', 'сырой', 'достаёте', 'розетка', 'смотрятся', 'болезнь', 'сброс', 'тихий проблема', 'поворот', 'опять ломаться', 'т.к', 'поддерживать', 'десять', 'вернуть', 'скоро', 'звонилка', 'ветер', 'прожить', 'сотовый связь', 'будто', 'гарантийный', 'другим', 'вдруг', 'количество', 'после', 'собираться', 'ладон

Получаем множества со словами только из положительных и только из отрицательных отзывов:

In [37]:
positive_set = positive_tokens.difference(negative_tokens)
negative_set = negative_tokens.difference(positive_tokens)
print(positive_set)
print(negative_set)

{'выпивать', 'возможность', 'отличный аккумулятор', 'лаконичный', 'правда', 'потеряю', 'первый смартфон', 'суперэкономного', 'китайский', 'полноценный', 'ощутимых', 'сомневаться', 'запилить', 'сдержанно', 'калькулятор', 'чат', 'интерфейс', 'решаться', 'текстовый редактор', 'юзаю', 'пр', 'поискать', 'трудный', 'шучу', 'лишь', 'цветной', 'умеренный', 'теоефон', 'последний прошивках', 'дружелюбный', 'надпись', 'значительно', 'неработает', 'здесь', 'всьо', 'кстати', 'плавненько', '472', 'халява', 'смарфон', 'общий сумма', 'нём', 'ночь', 'неэстетично', 'четко', 'зернистость', 'лаг', 'отлично рекомендовать', 'подвисаний', 'бюджетник', 'заказать', 'добротно', 'отличный соотношение', 'адекватный цена', 'хороший смартфон', 'девайс', 'оправдывать', 'текст', 'чужой', 'почитать', 'отдельно хотеться', 'айфон', 'бракованый теоефон', 'классный телефон', 'глючить.единственное прошивка', 'дорогой телефон', 'поилой', 'первый впечатление', 'небольшой поправка', 'мениатюрный', 'видный', 'перед', 'надо пок

**Работа с тестовой выборкой**

In [39]:
def model_checker(test_base):
  answers = []
  for text in test_base:
    doc_t = nlp(text.lower())
    token_list_t = []
    pos_tags = []
    for token in doc_t:
      if token.is_alpha or token.like_num or not token.is_punct:
        token_list_t.append(token.lemma_)
        pos_tags.append(token.pos_) # new

    pos_tags.append(' ')
        # new
    upd_list = []
    for i, tag in enumerate(pos_tags):
      if tag == 'ADJ':
        if 'NOUN' in pos_tags[i + 1]:
          collocation = token_list_t[i] + ' ' + token_list_t[i + 1]
          upd_list.append(collocation)

    upd_list_2 = []
    for i, tag in enumerate(pos_tags):
      if tag == 'ADV':
        if 'VERB' in pos_tags[i + 1]:
          collocation = token_list_t[i] + ' ' + token_list_t[i + 1]
          upd_list_2.append(collocation)


    new_list = token_list_t + upd_list + upd_list_2 
    # end of new block

    tokens = set(new_list)

    positivity = len(tokens.intersection(positive_set))
    negativity = len(tokens.intersection(negative_set))
    if positivity == 0 and negativity == 0:
      answers.append(2)
    if positivity > negativity:
      answers.append(1)
    if positivity < negativity:
      answers.append(0)

  return answers

model_checker(check_base)

[1, 1, 2, 0, 1, 0, 1, 0, 1, 1]

accuracy

In [27]:
from sklearn.metrics import accuracy_score
y_pred = model_checker(check_base)
y_true = right_answers
accuracy_score(y_true, y_pred)

0.5

сейчас посчитаем accuracy  после апдейта

In [40]:
from sklearn.metrics import accuracy_score
y_pred = model_checker(check_base)
y_true = right_answers
accuracy_score(y_true, y_pred)

0.5

Она не поменялась..

**UPD!!!**

Я нашла, что было не так с предыдущей домашкой:
1) Проблема с лемматизацией немного решилась после смены модели, используемой spasy
2) Главная проблема была не в лемматизации, а в том, что в функции работающей с тестовой выборкой было перепутано название списка, куда должны были записываться токены. Поэтому список токенов из тестовых отзывов оказался пустым, вследствие чего accuracy была нулевая, теперь она 0,5, что пока не идеально, но уже лучше..

В код сверху вставлены новые куски для дз 2, они отмечены комментариями + проведено сравнение качества предсказания до и после обновления